In [1]:
#Exercise 1
#This data structure is the basic system for Relation. 
import time
class Relation:
    class smallBlock:
        def __init__(self,size):
            self.size = size
            self.tuples = [None] * size
            self.numtuples = 0
        def __getitem__(self,idx):
            return self.tuples[idx]
        def __setitem__(self,idx,t):
            self.tuples[idx] = t
        def getnumtuples(self):
            return self.numtuples
        def insert(self,t):
            for i in range(self.size):
                if self.tuples[i] == None:
                    self.tuples[i] = t
                    self.numtuples += 1
                    return i
            raise IndexError("The block is full.")

        def retrieve(self,idx):
            return self.tuples[idx]
        
        #Check whether the small block is full.
        def isfull(self):
            count=0
            for i in range(self.size):
                if self.tuples[i] != None:
                    count +=1
            if count==self.size-1:
                return True
            return False

    #Placeholder class.
    class __Placeholder:
        def __init__(self):
            pass
        def __eq__(self,other):
            return False

    #For the fill factor, we set min is 1/2, max is 1.
    def __init__(self,thesize,blocksize,key):
        self.size=thesize
        self.elements=[None]*self.size
        self.blocksize=blocksize
        self.key=key
        for i in range(0,self.size//2):
            #The initialization part of the function. The first block is for overflow block, the second one is main data block.
            self.elements[i]=[Relation.smallBlock(self.blocksize),Relation.smallBlock(self.blocksize)]  

    #insert data operation.
    def insert(self,t):
        #create the pointer
        idx1=t[0]%self.blocksize
        idx2=t[0]//self.blocksize
        if self.elements[idx2][0].isfull()== False:
            self.elements[idx2][0][idx1]=t
            self.elements[idx2][0].numtuples+=1
        if self.elements[idx2][0].isfull()== True:
            self.sorting(idx2)
            self.split(self.elements[idx2][1],idx2)
        return self.findpointer(t)
        
    #delete the element represented by data t.
    def deletion(self,t):
        flag=0
        for i in self.elements:
            if i != None:
                for j in i[0].tuples:
                    if t==j:
                        flag=1
                        num=i[0].tuples.index(j)
                        #Place there a class placeholder.
                        i[0].tuples[num]=Relation.__Placeholder()
                        i[0].numtuples-=1
                        if i[0].numtuples ==i[0].size//2:
                            self.merge(i)
                        return True

                for k in i[1].tuples:
                    if t==k:
                        flag=1
                        num=i[1].tuples.index(k)
                        i[1].tuples[num]=Relation.__Placeholder()
                        i[1].numtuples-=1
                        if i[1].numtuples ==i[1].size//2:
                            self.merge(i)
                        return True
        if flag==1:
            return True
            
        if flag==0:
            raise IndexError('No such key')
    
    #delete the element by key.
    def keydelete(self,key):
        element=self.find(key)
        self.deletion(element)


    #Sorting method, to sort the block marked in idx2. Fulfill the main block, and sort it.
    def sorting(self,idx2):
        list0=[]
        for i in self.elements[idx2][0].tuples:
            if i!=None and type(i)!=Relation.__Placeholder:
                list0.append(i)
        for i in self.elements[idx2][1].tuples:
            if i!=None and type(i)!=Relation.__Placeholder:
                list0.append(i)

        #Lots of differnet keys, so we need prepare different key sort method to primary key.
        if self.key=='Movie':
            list0.sort(key=lambda x:(x[1],x[2]))

        if self.key=='Person':
            list0.sort(key=lambda x:x[1])
        
        if self.key=='Award':
            list0.sort(key=lambda x:x[1])
        
        if self.key=='Restriction_Category':
            list0.sort(key=lambda x:(x[1],x[2]))
        
        if self.key=='Director':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Writer':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Crew':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Role':
            list0.sort(key=lambda x:(x[2],x[3],x[4]))
        
        if self.key=='Scene':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Restriction':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Appearance':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Movie_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Crew_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Director_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3]))
        
        if self.key=='Writer_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))
        
        if self.key=='Actor_Award':
            list0.sort(key=lambda x:(x[1],x[2],x[3],x[4]))

        self.elements[idx2][0]=Relation.smallBlock(self.blocksize)
        self.elements[idx2][1]=Relation.smallBlock(self.blocksize)
        for j in list0:
            self.elements[idx2][1].insert(j)
        
    
    #Split part
    def split(self,elements,idx):
        print('split call')
        for i in range(self.size//2,self.size):
            if self.elements[i]==None:
                self.elements[i]=[Relation.smallBlock(self.blocksize),Relation.smallBlock(self.blocksize)]
                for j in range(self.elements[i][0].size//2,self.elements[i][0].size):
                    self.elements[i][0].insert(elements[j])
                    self.elements[idx][1][j]=None
                    self.elements[idx][1].numtuples-=1
                return i 
                    


    #Merge operation, to combine two sparse blocks into one block.
    def merge(self,tuplea):
        list1=[]
        print('merge call')
        for i in tuplea[0].tuples:
            if i!= None:
                list1.append(i)
        for i in tuplea[1].tuples:
            if i!= None:
                list1.append(i)
        for i in range(0,self.size):
            if self.elements[i]==tuplea:
                if i==0:
                    for k in list1:
                        self.elements[1][0].insert(k)
                    #Because there are multi-choices for the fill task at the original place after merge operation
                    #We choose to place there an new block.
                    self.elements[i]=[Relation.smallBlock(self.blocksize),Relation.smallBlock(self.blocksize)]
                    return
                if i==self.size-1:
                    for k in list1:
                        self.elements[self.size-1][0].insert(k)
                    self.elements[i]=[Relation.smallBlock(self.blocksize),Relation.smallBlock(self.blocksize)]
                    return
                if self.elements[i-1][1].numtuples>=self.elements[i+1][1].numtuples:
                    for k in list1:
                        self.elements[i+1][0].insert(k)
                    self.elements[i]=[Relation.smallBlock(self.blocksize),Relation.smallBlock(self.blocksize)]
                    return
                if self.elements[i-1][1].numtuples < self.elements[i+1][1].numtuples:
                    for k in list1:
                        self.elements[i-1][0].insert(k)
                    self.elements[i]=[Relation.smallBlock(self.blocksize),Relation.smallBlock(self.blocksize)]
                    return

    #This function is same as find/ another kind of retireval.
    def find(self,num):
        #Linear search
        for i in self.elements:
            if i != None:
                for j in i[0].tuples:
                    if j!=None and type(j) != Relation.__Placeholder and num==j[0]:
                        return j
                couplelist=[]
                for j in i[1].tuples:
                    if j!= None and type(j)!=Relation.__Placeholder:
                        couplelist.append(j)
                if couplelist ==[]:
                    continue
                couplelist.sort(key=lambda x:x[0])
                #binary search part 
                left=0
                right=len(couplelist)
                while left<=right:
                    mid=(left+right)//2
                    if num<couplelist[mid-1][0]:
                        right=mid-1
                    elif num>couplelist[mid-1][0]:
                        left=mid+1
                    else:
                        return couplelist[mid-1]
        return -1
    
    #Find the data using pointer. (another kind of retrieval operation)
    def retrievedirectly(self,pointer):
        blocknumber = pointer[0]
        idx = pointer[1]
        blocktype=pointer[2]
        return self.elements[blocknumber][blocktype][idx]
    
    #Find the pointer corresponding to the data.
    def findpointer(self,t):
        for i in range(len(self.elements)):
            if self.elements[i] != None:
                for j in range(self.elements[i][0].size):
                    if self.elements[i][0][j]==t:
                        return (i,j,0)
                for j in range(self.elements[i][1].size):
                    if self.elements[i][1][j]==t:
                        return (i,j,1)

In [4]:
#test
#For movie, primary key is (m,y), secondary key will be the first element(index in lecture slides).
#We change the size, also show the correctness of split operation and merge operation.
#We set the size of blocks as 20, to test merge and split operation.
#Use movie relation as one example.
Movie=Relation(100,20,'Movie')

Movie.insert((1,'Titanic',1997,'USA',195,'romance'))#Movie(m, y, c, t, g)
Movie.insert((2,'Shakespeare in Love',1998,'UK',122,'romance'))#Movie(m, y, c, t, g)
Movie.insert((3,'The Cider House Rules',1999,'USA',125,'drama'))#Movie(m, y, c, t, g)
Movie.insert((4,'Gandhi',1982,'India',188,'drama'))#Movie(m, y, c, t, g)
Movie.insert((5,'American Beauty',1999,'USA',121,'drama'))#Movie(m, y, c, t, g)
Movie.insert((6,'Affliction',1997,'USA',113,'drama'))#Movie(m, y, c, t, g)
Movie.insert((7,'Life is Beautiful',1997,'Italy',118,'comedy'))#Movie(m, y, c, t, g)
Movie.insert((8,'Boys Dont Cry',1999,'USA',118,'drama'))#Movie(m, y, c, t, g)
Movie.insert((9,'Saving Private Ryan',1998,'USA',170,'action'))#Movie(m, y, c, t, g)
Movie.insert((10,'The Birds',1963,'USA',119,'horror'))#Movie(m, y, c, t, g)
Movie.insert((11,'The Matrix',1999,'USA',136,'action'))#Movie(m, y, c, t, g)
Movie.insert((12,'Toy Story',1995,'USA',81,'animation'))#Movie(m, y, c, t, g)
Movie.insert((13,'You have Got Mail',1998,'USA',119,'comedy'))#Movie(m, y, c, t, g)
Movie.insert((14,'Proof of Life',2000,'USA',135,'drama'))#Movie(m, y, c, t, g)
Movie.insert((15,'Hanging Up',2000,'USA',94,'comedy'))#Movie(m, y, c, t, g)
Movie.insert((16,'The Price of Milk',2000,'New Zealand',87,'romance'))#Movie(m, y, c, t, g)
Movie.insert((17,'The Footstep Man',1992,'New Zealand',89,'drama'))#Movie(m, y, c, t, g)
Movie.insert((18,'Topless Women Talk About Their Lives',1997,'New Zealand',87,'drama'))#Movie(m, y, c, t, g)


print('before split')
beg=time.time()
print(Movie.findpointer((18,'Topless Women Talk About Their Lives',1997,'New Zealand',87,'drama')))
end=time.time()
print('time cost',end-beg)
Movie.insert((19,'The Piano',1993,'New Zealand',121,'romance'))#Movie(m, y, c, t, g)
Movie.insert((20,'Mad Max',1979,'Australia',88,'action'))#Movie(m, y, c, t, g)
print('After split')
print(Movie.findpointer((18,'Topless Women Talk About Their Lives',1997,'New Zealand',87,'drama')))

Movie.insert((21,'Strictly Ballroom',1992,'Australia',94,'comedy'))#Movie(m, y, c, t, g)
Movie.insert((22,'My Mother Frank',2000,'Australia',95,'comedy'))#Movie(m, y, c, t, g)
Movie.insert((23,'American Psycho',2000,'Canada',101,'horror'))#Movie(m, y, c, t, g)
Movie.insert((24,'Scream 2',1997,'USA',116,'horror'))#Movie(m, y, c, t, g)
Movie.insert((25,'Scream 3',2000,'USA',116,'horror'))#Movie(m, y, c, t, g)
Movie.insert((26,'Traffic',2000,'Germany',147,'crime'))#Movie(m, y, c, t, g)
Movie.insert((27,'Psycho',1960,'USA',109,'horror'))#Movie(m, y, c, t, g)
Movie.insert((28,'I Know What You Did Last Summer',1997,'USA',100,'horror'))#Movie(m, y, c, t, g)
Movie.insert((29,'Cruel Intentions',1999,'USA',95,'romance'))#Movie(m, y, c, t, g)
Movie.insert((30,'Wild Things',1998,'USA',108,'crime'))#Movie(m, y, c, t, g)

print('before merge')
print(Movie.findpointer((20,'Mad Max',1979,'Australia',88,'action')))
Movie.deletion((21,'Strictly Ballroom',1992,'Australia',94,'comedy'))
print('after merge')
print(Movie.findpointer((20,'Mad Max',1979,'Australia',88,'action')))

Movie.insert((31,'Alien',1979,'UK',117,'horror'))#Movie(m, y, c, t, g)
Movie.insert((32,'Aliens',1986,'USA',154,'action'))#Movie(m, y, c, t, g)
Movie.insert((33,'Alien 3',1992,'USA',115,'action'))#Movie(m, y, c, t, g)
Movie.insert((34,'Alien: Resurrection',1997,'USA',109,'action'))#Movie(m, y, c, t, g)
Movie.insert((35,'Gladiator',2000,'UK',154,'drama'))#Movie(m, y, c, t, g)
Movie.insert((36,'The World Is Not Enough',1999,'UK',127,'action'))#Movie(m, y, c, t, g)
Movie.insert((37,'Heat',1995,'USA',171,'action'))#Movie(m, y, c, t, g)
Movie.insert((38,'American History X',1998,'USA',119,'drama'))#Movie(m, y, c, t, g)
Movie.insert((39,'Fight Club',1999,'Germany',139,'drama'))#Movie(m, y, c, t, g)
Movie.insert((40,'Out of Sight',1998,'USA',122,'comedy'))#Movie(m, y, c, t, g)

#Movie.sorting(0) #use to test the correctness of sorting. It offically is used in Insert operation.
Movie.deletion((34,'Alien: Resurrection',1997,'USA',109,'action'))
Movie.sorting(0)
print('test find')
beg=time.time()
print(Movie.find(40))#Find the data
end=time.time()
print('time cost', end-beg)
#Movie.deletion((34,'Alien: Resurrection',1997,'USA',109,'action')) Repeat delete operation will raise error

before split
(0, 18, 0)
time cost 0.0002288818359375
split call
After split
(50, 6, 0)
before merge
(1, 0, 0)
merge call
after merge
(2, 0, 0)
test find
(40, 'Out of Sight', 1998, 'USA', 122, 'comedy')
time cost 0.0002129077911376953
